# Disease prediction based on ensemble machine learning model

## Input：
merge_asthma_train.bed merge_asthma_train.bim merge_asthma_train.fam

merge_asthma_valid.bed merge_asthma_valid.bim merge_asthma_valid.fam      

merge_asthma_test.bed merge_asthma_test.bed merge_asthma_test.bed

**“asthma” in the filename is the disease name (variable dis in the code), you can change anyword you like**
## output: 
AUC and prediction score of ensemble model



## Step 1: Calculate SNP associations from training set

1. Plink (https://www.cog-genomics.org/plink/1.9/) should be installed first and add the path in ~/.bashrc 
2. covarfile is the file name with covariats, you can find the format and more information in https://www.cog-genomics.org/plink/1.9/

In [ ]:
dis='asthma'
import os
cmd="plink --adjust --bfile merge_"+dis+"_train --covar covarfile --logistic --out "+dis+"_train_assoc"
os.system(cmd)


## Step 2: Generate SNP list based on four p-value thresholds

In [ ]:
import os
dis='asthma'
infile1=open(dis+'_train_assoc.assoc.logistic.adjusted')
snplist1=open(dis+'_5E3_SNP','w')
snplist2=open(dis+'_5E4_SNP','w')
snplist3=open(dis+'_5E5_SNP','w')
snplist4=open(dis+'_5E6_SNP','w')
SNP1=0
SNP2=0
SNP3=0
SNP4=0
linenum=0
for line in infile1:
    if linenum>0:
        A=line.strip('\n').rsplit()
        if float(A[3])<0.005:
            snplist1.write(A[1]+'\n')
            SNP1+=1
        if float(A[3])<0.0005:
            snplist2.write(A[1]+'\n')
            SNP2+=1
        if float(A[3])<0.00005:
            snplist3.write(A[1]+'\n')
            SNP3+=1
        if float(A[3])<0.000005:
            snplist4.write(A[1]+'\n')
            SNP4+=1
    linenum+=1
infile1.close()
snplist1.close()
snplist2.close()
snplist3.close()
snplist4.close()

## Step 3: LD prunning and extract the remaining SNPs from train, validation and test datasets

In [ ]:
dis='asthma'
for data in ['train','valid','test']:
    os.system('plink --bfile merge_'+dis+'_'+data+' --extract '+dis+'_5E3_SNP --make-bed --out '+dis+'_5E3_'+data+'_extract\n')
    os.system('plink --bfile merge_'+dis+'_'+data+' --extract '+dis+'_5E4_SNP --make-bed --out '+dis+'_5E4_'+data+'_extract\n')
    os.system('plink --bfile merge_'+dis+'_'+data+' --extract '+dis+'_5E5_SNP --make-bed --out '+dis+'_5E5_'+data+'_extract\n')
    os.system('plink --bfile merge_'+dis+'_'+data+' --extract '+dis+'_5E6_SNP --make-bed --out '+dis+'_5E6_'+data+'_extract\n')

    os.system('plink --bfile '+dis+'_5E6_train_extract --indep-pairwise 50 5 0.5 --out '+data+'_5E6_'+dis+'\n')
    os.system('plink --bfile '+dis+'_5E5_train_extract --indep-pairwise 50 5 0.5 --out '+data+'_5E5_'+dis+'\n')
    os.system('plink --bfile '+dis+'_5E4_train_extract --indep-pairwise 50 5 0.5 --out '+data+'_5E4_'+dis+'\n')
    os.system('plink --bfile '+dis+'_5E3_train_extract --indep-pairwise 50 5 0.5 --out '+data+'_5E3_'+dis+'\n')


    os.system('plink --bfile '+dis+'_5E6_'+data+'_extract --prune --extract '+data+'_5E6_'+dis+'.prune.in --make-bed --out '+dis+'_'+data+'_5E6_SNP')
    os.system('plink --bfile '+dis+'_5E5_'+data+'_extract --prune --extract '+data+'_5E5_'+dis+'.prune.in --make-bed --out '+dis+'_'+data+'_5E5_SNP')
    os.system('plink --bfile '+dis+'_5E4_'+data+'_extract --prune --extract '+data+'_5E4_'+dis+'.prune.in --make-bed --out '+dis+'_'+data+'_5E4_SNP')
    os.system('plink --bfile '+dis+'_5E3_'+data+'_extract --prune --extract '+data+'_5E3_'+dis+'.prune.in --make-bed --out '+dis+'_'+data+'_5E3_SNP')
    
    os.system('plink --bfile '+dis+'_'+data+'_5E6_SNP --prune --recodeA --out '+dis+'_'+data+'_5E6_recodeA')
    os.system('plink --bfile '+dis+'_'+data+'_5E5_SNP --prune --recodeA --out '+dis+'_'+data+'_5E5_recodeA')
    os.system('plink --bfile '+dis+'_'+data+'_5E4_SNP --prune --recodeA --out '+dis+'_'+data+'_5E4_recodeA')
    os.system('plink --bfile '+dis+'_'+data+'_5E3_SNP --prune --recodeA --out '+dis+'_'+data+'_5E3_recodeA')

## Step 4: Recode genotype and L&E into .npy files

### Notice
Three files with L&E information (named as **L&E_train.matrix**, **L&E_valid.matrix** and **L&E_test.matrix**) should be prepared. 

Each row is a L&E and each column is an individual.

In [ ]:
from collections import defaultdict
import matplotlib
import re
import numpy as np
import operator
from scipy import stats as sta
import math
import multiprocessing
import operator
import scipy
import os
from itertools import groupby
from operator import itemgetter 
def generate_G_P(path_diseaseid,path_data):
    phenotype=np.genfromtxt(path_diseaseid,dtype=str) 
    data=np.genfromtxt(path_data,dtype=str)
    SNPindexlist=[]
    SNP_id=[]
    ind_id=[]
    indindexlist=[]
    ind_genotype=[]
    all_disease=[]
    totalind=phenotype.shape[0]
    print(totalind)
    for i in range(totalind):
        print(i)
        if i==0:
            snpnum=data[0].shape[0]
            for j in range(6,snpnum):
                SNPindexlist.append(j)
                SNP_id.append(data[0][j].split('_')[0])
        else:
            if len(SNPindexlist)==1:
                tmp=data[i][6]
                if tmp=='NA':
                    tmp='0'
            else:
                tmp=list(itemgetter(*SNPindexlist)(data[i]))
                for x in range(len(tmp)):
                    if tmp[x]=='NA':
                        tmp[x]='0'
            indindexlist.append(i-1)
            ind_genotype.append(list(map(int,tmp)))   
            ind_id.append(phenotype[i-1][0])
            all_disease.append(int(data[i][5])-1)
    all_genotype=sta.zscore(ind_genotype)
    s=np.isnan(all_genotype)
    all_genotype[s]=1
    return all_genotype,all_disease,ind_id,SNP_id

dis='asthma'
os.system('mkdir train')
os.system('mkdir valid')
os.system('mkdir test')
outpath1='./train/'
outpath2='./valid/'
outpath3='./test/'
for i in ['5E3','5E4','5E5','5E6']:
    if os.path.exists(dis+'_train_'+i+'_recodeA.raw')==False:
        continue
    out_train_disease=outpath1+str(dis)+'_disease_train'
    out_valid_disease=outpath2+str(dis)+'_disease_valid'
    out_test_disease=outpath3+str(dis)+'_disease_test'
     
    out_geno_train=outpath1+str(dis)+'_genotype_train_'+i
    out_geno_valid=outpath2+str(dis)+'_genotype_valid_'+i
    out_geno_test=outpath3+str(dis)+'_genotype_test_'+i
  
    out_snpid=outpath1+str(dis)+'_snpid'

    out_train_id=outpath1+str(dis)+'_indid_train'
    out_valid_id=outpath2+str(dis)+'_indid_valid'
    out_test_id=outpath3+str(dis)+'_indid_test'


    [train_genotype,train_disease,train_indid,train_SNPid]=generate_G_P(dis+'_train_'+i+'_SNP.fam',dis+'_train_'+i+'_recodeA.raw')
    [valid_genotype,valid_disease,valid_indid,valid_SNPid]=generate_G_P(dis+'_valid_'+i+'_SNP.fam',dis+'_valid_'+i+'_recodeA.raw')
    [test_genotype,test_disease,test_indid,test_SNPid]=generate_G_P(dis+'_test_'+i+'_SNP.fam',dis+'_test_'+i+'_recodeA.raw')

    np.save(out_snpid,train_SNPid)
    np.save(out_geno_train,train_genotype)
    np.save(out_geno_valid,valid_genotype)
    np.save(out_geno_test,test_genotype)
    np.save(out_train_id,train_indid)
    np.save(out_valid_id,valid_indid)
    np.save(out_test_id,test_indid)
    np.save(out_train_disease,train_disease)
    np.save(out_valid_disease,valid_disease)
    np.save(out_test_disease,test_disease)
out_train_pheno=outpath1+str(dis)+'_phenotype_train'
out_valid_pheno=outpath2+str(dis)+'_phenotype_valid'
out_test_pheno=outpath3+str(dis)+'_phenotype_test'
for data in ["train","valid","test"]:
    self_phenotype=np.genfromtxt('L&E_'+data+'.matrix',dtype=float)
    num_L=self_phenotype.shape[0]
    for j in range(num_L):
        var=np.var(self_phenotype[j])
        if var!=0:
            all_phenotype.append(sta.zscore(self_phenotype[j]))
        else:
            all_phenotype.append(self_phenotype[j])
    if data=='train':
        np.save(out_train_pheno,all_phenotype)
    elif data=='valid':
        np.save(out_valid_pheno,all_phenotype)
    else:
        np.save(out_test_pheno,all_phenotype)

## Step 5: Generate 80 candidate models from Neural Network (NN), adaboost (Ada),Gradient Boosting (GB), Lasso Regression (LR), Random Forest (RF)

In the ML folder, we will get the prediction scores for the individuals in validation and test sets. Each file includes two column, 

**Validation set:**

*_NN_valid.npy, *_ada_valid.npy, *_GB_valid.npy, *_LR_valid.npy and *_RF_valid.npy 

**Test set:**

*_NN_test.npy, *_ada_test.npy, *_GB_test.npy, *_LR_test.npy and *_RF_test.npy 


In [ ]:
from collections import defaultdict
import matplotlib
import os
import numpy as np
import operator
from scipy import stats as sta
import math
from scipy.interpolate import spline
import operator
from itertools import groupby
from sklearn.metrics import roc_auc_score
from operator import itemgetter
from sklearn.linear_model import LogisticRegression
from scipy.stats.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from scipy.stats import norm
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import multiprocessing
import pickle
from  sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

dir_path1='./train/'
dir_path2='./valid/'
dir_path3='./test/'
os.system('./ML')
dis='asthma'

disease_train=np.load(dir_path1+dis+'_disease_train.npy')
Cxlist_NN=[(20,20,20),(30,30),(10,10,10,10),(30,20,10)]
Cxlist_ada=[DecisionTreeClassifier(),LogisticRegression(),ExtraTreeClassifier(),GaussianNB()]
Cxlist_GB=[0.0001,0.001,0.01,0.1]
Cxlist_LR=[0.0001,0.001,0.01,0.1]
Cxlist_RF=[0.0001,0.001,0.01,0.1]
for s in [1,2,3,4]:
    if s==1:
        Geno_train=np.load(dir_path1+dis+'_genotype_train_5E3.npy')
        Geno_valid=np.load(dir_path2+dis+'_genotype_valid_5E3.npy')
        Geno_test=np.load(dir_path3+dis+'_genotype_test_5E3.npy')
    elif s==2:
        Geno_train=np.load(dir_path1+dis+'_genotype_train_5E4.npy')
        Geno_valid=np.load(dir_path2+dis+'_genotype_valid_5E4.npy')
        Geno_test=np.load(dir_path3+dis+'_genotype_test_5E4.npy')
    elif s==3:
        Geno_train=np.load(dir_path1+dis+'_genotype_train_5E5.npy')
        Geno_valid=np.load(dir_path2+dis+'_genotype_valid_5E5.npy')
        Geno_test=np.load(dir_path3+dis+'_genotype_test_5E5.npy')
    else:
        Geno_train=np.load(dir_path1+dis+'_genotype_train_5E6.npy')
        Geno_valid=np.load(dir_path2+dis+'_genotype_valid_5E6.npy')
        Geno_test=np.load(dir_path3+dis+'_genotype_test_5E6.npy')

    for t in [1,2,3,4]:
        Cx_NN=Cxlist_NN[t-1]
        NN=MLPClassifier(hidden_layer_sizes=Cx_NN,max_iter=1000)
        NN.fit(Geno_train,disease_train)
        Y=NN.predict_proba(Geno_valid)
        np.save('./ML/'+dis+'_'+str(s)+'_'+str(t)+'_NN_valid.npy',Y[:,1])
        Y=NN.predict_proba(Geno_test)
        np.save('./ML/'+dis+'_'+str(s)+'_'+str(t)+'_NN_test.npy',Y[:,1])

        Cx_ada=Cxlist_ada[t-1]
        ada = AdaBoostClassifier(base_estimator=Cx_ada)
        ada.fit(Geno_train,disease_train)
        Y=ada.predict_proba(Geno_valid)
        np.save('./ML/'+dis+'_'+str(s)+'_'+str(t)+'_ada_valid.npy',Y[:,1])
        Y=ada.predict_proba(Geno_test)
        np.save('./ML/'+dis+'_'+str(s)+'_'+str(t)+'_ada_test.npy',Y[:,1])

        Cx_GB=Cxlist_GB[t-1]
        GB = GradientBoostingClassifier(min_impurity_decrease=Cx_GB)
        GB.fit(Geno_train,disease_train)
        Y=GB.predict_proba(Geno_valid)
        np.save('./ML/'+dis+'_'+str(s)+'_'+str(t)+'_GB_valid.npy',Y[:,1])
        Y=GB.predict_proba(Geno_test)
        np.save('./ML/'+dis+'_'+str(s)+'_'+str(t)+'_GB_test.npy',Y[:,1])

        Cx_LR=Cxlist_LR[t-1]
        LR = LogisticRegression(penalty='l1', C=Cx_LR, max_iter=10000)
        LR.fit(Geno_train,disease_train)
        Y=LR.predict_proba(Geno_valid)
        np.save('./ML/'+dis+'_'+str(s)+'_'+str(t)+'_LR_valid.npy',Y[:,1])
        Y=LR.predict_proba(Geno_test)
        np.save('./ML/'+dis+'_'+str(s)+'_'+str(t)+'_LR_test.npy',Y[:,1])

        Cx_RF=Cxlist_RF[t-1]
        RF = RandomForestClassifier(min_impurity_decrease=Cx_RF)
        RF.fit(Geno_train,disease_train)
        Y=RF.predict_proba(Geno_valid)
        np.save('./ML/'+dis+'_'+str(s)+'_'+str(t)+'_RF_valid.npy',Y[:,1])
        Y=RF.predict_proba(Geno_test)
        np.save('./ML/'+dis+'_'+str(s)+'_'+str(t)+'_RF_test.npy',Y[:,1])
 


## Step 6: calculate disease prediction score by ensemble model by integrating 80 models and AUC in the test set 

This step will generate three files as output:

***_valid_ensemble.npy**: disease prediction scores by ensemble model for validation set

***_valid_ensemble.npy**: disease prediction scores by ensemble model for test set

**ensemble**: the value of AUC for test set

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
dir1='./valid/'
dir2='./test/'
dis='asthma'
out=open('ensemble','w')
disease_valid=np.load(dir1+dis+'_disease_valid.npy')
disease_test=np.load(dir2+dis+'_disease_test.npy')
for i in ['GB','NN','ada','RF','LR']:
    if i=='GB':
        filename1=dis+'_1_1_'+i+'_valid.npy'
        filename2=dis+'_1_1_'+i+'_test.npy'
        data1=np.load(filename1)
        data2=np.load(filename2)
        enseinput=data1.reshape(-1,1)
        ensetest=data2.reshape(-1,1)
    for m in ['1','2','3','4']:
        for n in ['1','2','3','4']:
            filename1=dis+'_'+m+'_'+n+'_'+i+'_valid.npy'
            filename2=dis+'_'+m+'_'+n+'_'+i+'_test.npy'
            data1=np.load(filename1)
            data2=np.load(filename2)
            if i=='GB' and m=='1' and n=='1':
                continue
            enseinput=np.concatenate((enseinput,data1.reshape(-1,1)),axis=1)
            ensetest=np.concatenate((ensetest,data2.reshape(-1,1)),axis=1)
LR = LogisticRegression(penalty='l1', max_iter=10000)
LR.fit(enseinput,disease_valid)
Y1=LR.predict_proba(enseinput)
Y2=LR.predict_proba(ensetest)
np.save(dis+'_valid_ensemble.npy',Y1[:,1]) 
np.save(dis+'_test_ensemble.npy',Y2[:,1])
AUC_test=roc_auc_score(disease_test,Y2[:,1])
out.write(dis+'\t'+str(AUC_test)+'\n')
out.close()

